In [1]:
import random
from datetime import datetime, timedelta
# Generate 100 flash cards
num_flash_cards = 100

flash_cards = []

for _ in range(num_flash_cards):

    # Simulate the number of times the card was correctly answered CONSECUTIVELY
    times_correctly_answered = random.randint(0, 4)# Randomly generate test_date
    
    random_number = random.random()
    
    if random_number <= 0.05:
            test_date = datetime.now() - timedelta(days=1)
    elif random_number <= 0.75:
            test_date = datetime.now()
    else:
            test_date = datetime.now() + timedelta(days=random.randint(1, 7))

    if test_date.date()  == datetime.now().date():
            probability = random.uniform(0, 1)
            if probability < 0.05:
                        mark = 0
            elif  probability < 0.55:
                   mark = 1
            else:
                 mark = -1 
    else: 
           mark = 0
    


    flash_card = ("times_correctly_answered", times_correctly_answered, "test_date", test_date, "mark", mark )
    flash_cards.append(flash_card)




import pandas as pd

# Extracting data for each column
times_correctly_answered = [card[1] for card in flash_cards]
test_dates = [card[3] for card in flash_cards]
marks = [card[5] for card in flash_cards]

# Creating a DataFrame
cards_df = pd.DataFrame({
    'times_correctly_answered': times_correctly_answered,
    'test_date': test_dates,
    'mark': marks,
})

cards_df['card'] = 'Card ' + (cards_df.index + 1).astype(str)


from IPython.display import display

print("The raw data and old schedule of the flash card")
display(cards_df)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
The raw data and old schedule of the flash card


,times_correctly_answered,test_date,mark,card
0,0,2023-12-01 21:41:50.535217,0,Card 1
1,4,2023-11-24 21:41:50.535241,-1,Card 2
2,1,2023-11-24 21:41:50.535247,-1,Card 3
3,3,2023-11-24 21:41:50.535249,1,Card 4
4,1,2023-11-24 21:41:50.535252,-1,Card 5
...,...,...,...,...
95,4,2023-11-24 21:41:50.535483,-1,Card 96
96,3,2023-11-24 21:41:50.535485,-1,Card 97
97,1,2023-11-24 21:41:50.535487,-1,Card 98
98,1,2023-11-24 21:41:50.535489,1,Card 99


In [2]:
from datetime import datetime, timedelta
import pandas as pd
import random


class ScheduleGenerator:
    
    def __init__(self, flash_cards, end_date_str):
        self.flash_cards = flash_cards
        self.today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        self.end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M:%S')
        self.time_range_seconds = (self.end_date - datetime.now()).total_seconds()
        print(self.time_range_seconds/(60*60*24))

    def generate_schedule(self):
        # Remove rows (completion) with times_correctly_answered equal to 5
        self.flash_cards = self.flash_cards[self.flash_cards['times_correctly_answered'] != 5]

        # Reset test dates that are before today to today
        self.flash_cards.loc[self.flash_cards['test_date'] < self.today, 'test_date'] = self.today

        for index, row in self.flash_cards.iterrows():
    
            if row['test_date'].date() == self.today.date():
               
            # Update times_correctly_answered based on mark
               if row['mark'] == 1: 
                   self.flash_cards.at[index, 'times_correctly_answered'] += 1
               elif (row['times_correctly_answered'] != 0) & (row['mark'] == -1): 
                   self.flash_cards.at[index, 'times_correctly_answered'] -= 1
               elif (row['times_correctly_answered'] == 0) & (row['mark'] == -1): 
                   self.flash_cards.at[index, 'times_correctly_answered'] = 0
            
                #adjust the progress bar
               if row['times_correctly_answered'] == 0: 
                   self.flash_cards.at[index, 'test_date'] += pd.to_timedelta(48, unit='h')
               elif row['times_correctly_answered'] == 1: 
                   self.flash_cards.at[index, 'test_date'] += pd.to_timedelta(120, unit='h')
               elif row['times_correctly_answered'] == 2: 
                   self.flash_cards.at[index, 'test_date'] += pd.to_timedelta(self.time_range_seconds/25, unit='s')
               elif row['times_correctly_answered'] == 3: 
                   self.flash_cards.at[index, 'test_date'] += pd.to_timedelta(self.time_range_seconds/5, unit='s')
               elif row['times_correctly_answered'] == 4: 
                   self.flash_cards.at[index, 'test_date'] += pd.to_timedelta(self.time_range_seconds/2, unit='s')

        self.flash_cards = self.flash_cards[self.flash_cards['times_correctly_answered'] != 5]
        self.flash_cards['test_date'] = self.flash_cards['test_date'].dt.date
        # Creating a DataFrame for visualization
        df = pd.DataFrame(self.flash_cards, columns=[
            'times_correctly_answered',
            'test_date',
            'card'
        ])

        return df




In [3]:
generator = ScheduleGenerator(cards_df, '2024-04-30 16:30:00')

# Generate the schedule
schedule_df = generator.generate_schedule()

# Display the resulting schedule DataFrame
print("The dynamic new schedule of the flash card")
display(schedule_df)

157.78312913631945
The dynamic new schedule of the flash card


,times_correctly_answered,test_date,card
0,0,2023-12-01,Card 1
1,3,2024-02-11,Card 2
2,0,2023-11-29,Card 3
3,4,2023-12-26,Card 4
4,0,2023-11-29,Card 5
...,...,...,...
95,3,2024-02-11,Card 96
96,2,2023-12-26,Card 97
97,0,2023-11-29,Card 98
98,2,2023-11-29,Card 99
